In [149]:
#Import Packages
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from citipy import citipy
import requests
import time
from datetime import datetime
import config

#API Key
#weather_api_key = 'a43a0b0f44bbe9b2ea0b0eecfa48b989'

#Saved File Destination
file_destination = r'C:\Users\mattg\Desktop\Columbia\Module-6\Weather_Database\WeatherPy_Database.csv'


In [150]:
#Functions

#Create Random List of Lat Longs. User Inputs how many random lat longs to include
def lat_lngs(size):
    lats = np.random.uniform(low=-90.000, high=90.000, size=size)
    lngs = np.random.uniform(low=-180.000, high=180.000, size=size)
    lat_lngs = zip(lats, lngs)
    return lat_lngs


In [151]:
#Random List of Coordinates
coordinates = list(lat_lngs(2000))
#print(coordinates)

In [152]:
#Lists To Hold Data

cities = []
country_codes = []
lat = []
long = []
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name 
    country = citipy.nearest_city(coordinate[0], coordinate[1]).country_code
    if city not in cities:
        cities.append(city)

len(cities)       

724

In [153]:
#Weather API City Data List

city_data = []
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

url_list = []

for city in cities:
    city_url = url + "&q=" + city.replace(" ","+")
    url_list.append(city_url)

for city_url in url_list:
    try:
        # Parse the JSON and retrieve data.
        
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_name = city_weather['name']
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        for i in city_weather['weather']:
            city_description = i['description']
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city_name,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                         'Current Description': city_description})
    except:
        pass

In [154]:
df = pd.DataFrame.from_dict(city_data)
df = df[['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Current Description']]
df.to_csv(file_destination)
df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Tsimmerbude,RU,54.6750,20.1347,47.97,74,100,17.58,overcast clouds
1,Rikitea,PF,-23.1203,-134.9692,74.57,78,85,10.07,light rain
2,São João da Barra,BR,-21.6403,-41.0511,75.29,76,7,15.86,clear sky
3,Faanui,PF,-16.4833,-151.7500,79.45,75,97,10.29,overcast clouds
4,Upernavik,GL,72.7868,-56.1549,17.08,83,45,4.38,scattered clouds
...,...,...,...,...,...,...,...,...,...
658,Basco,PH,20.4487,121.9702,81.68,81,99,19.91,overcast clouds
659,Saraland,US,30.8207,-88.0706,69.80,48,18,7.90,few clouds
660,Kichmengskiy Gorodok,RU,59.9816,45.7854,45.88,88,100,14.20,overcast clouds
661,Svetlaya,RU,46.5389,138.3297,40.75,69,53,3.83,broken clouds
